In [1]:
from neuron import h
import numpy as np
soma = h.Section(name='soma')
soma.nseg = 10
soma.Ra = 100
soma.L = 1000
diam = np.linspace(1,10,11)
for i,seg in enumerate(soma.allseg()):
    seg.diam = diam[i-1]
x = np.linspace(0, 100, 10)
print(np.size(x))
y = 17 * np.sin(x)
z = 17 * np.cos(x)
xvec = h.Vector(x)
yvec = h.Vector(y)
zvec = h.Vector(z)
dvec = h.Vector([1] * len(x))
connections=np.zeros([len(x)-1,2])
connections[:,0]=np.linspace(1,len(x)-1,len(x)-1)
connections[:,1]=np.linspace(0,len(x)-2,len(x)-1)


h.pt3dadd(xvec, yvec, zvec, dvec, sec=soma)

soma.insert('hh')
for i,seg in enumerate(soma):
    seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
    seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
    seg.hh.gl = 0.0003    # Leak conductance in S/cm2
    seg.hh.el = -54.3     # Reversal potential in mV
#stim = h.IClamp(soma(0.5))
#stim.delay = 10
#stim.dur = 1
#stim.amp = 0.001

v_vec = h.Vector()        # Membrane potential vector
t_vec = h.Vector()        # Time stamp vector
v_vec.record(soma(0.5)._ref_v)
t_vec.record(h._ref_t)
simdur = 25.0
h.load_file('stdrun.hoc')
h.finitialize(-70)
h.continuerun(40)
h.topology()

10

|----------|       soma(0-1)



1.0

'Accent' or 'Blues' or 'BrBG' or 'BuGn' or 'BuPu' or 'CMRmap' or 'Dark2' or 'GnBu' or 'Greens' or 'Greys' or 'OrRd' or 'Oranges' or 'PRGn' or 'Paired' or 'Pastel1' or 'Pastel2' or 'PiYG' or 'PuBu' or 'PuBuGn' or 'PuOr' or 'PuRd' or 'Purples' or 'RdBu' or 'RdGy' or 'RdPu' or 'RdYlBu' or 'RdYlGn' or 'Reds' or 'Set1' or 'Set2' or 'Set3' or 'Spectral' or 'Vega10' or 'Vega20' or 'Vega20b' or 'Vega20c' or 'Wistia' or 'YlGn' or 'YlGnBu' or 'YlOrBr' or 'YlOrRd' or 'afmhot' or 'autumn' or 'binary' or 'black-white' or 'blue-red' or 'bone' or 'brg' or 'bwr' or 'cool' or 'coolwarm' or 'copper' or 'cubehelix' or 'file' or 'flag' or 'gist_earth' or 'gist_gray' or 'gist_heat' or 'gist_ncar' or 'gist_rainbow' or 'gist_stern' or 'gist_yarg' or 'gnuplot' or 'gnuplot2' or 'gray' or 'hot' or 'hsv' or 'inferno' or 'jet' or 'magma' or 'nipy_spectral' or 'ocean' or 'pink' or 'plasma' or 'prism' or 'rainbow' or 'seismic' or 'spectral' or 'spring' or 'summer' or 'terrain' or 'viridis' or 'winter'

In [4]:
from mayavi import mlab

points = mlab.pipeline.scalar_scatter(x, y, z, diam[1:]/2.0)
dataset = points.mlab_source.dataset
dataset.point_data.get_array(0).name = 'diameter'
dataset.lines = connections
dataset.point_data.update()
# The tube
src = mlab.pipeline.set_active_attribute(points, point_scalars='diameter')
stripper = mlab.pipeline.stripper(src)


tube = mlab.pipeline.tube(stripper, tube_sides = 6, tube_radius = 1)
tube.filter.capping = True
#        tube.filter.use_default_normal = False
tube.filter.vary_radius = 'vary_radius_by_absolute_scalar'

tube.children = [] # Removing the old ones 

potential = np.linspace(1,91,len(x))
array_id = dataset.point_data.add_array(potential)
dataset.point_data.get_array(array_id).name = 'voltage'
dataset.point_data.update()
src2 = mlab.pipeline.set_active_attribute(tube, 
                                            point_scalars='voltage')
surf = mlab.pipeline.surface(src2, colormap='gist_heat')

cb=mlab.colorbar(object=surf,nb_labels=10)
lut=surf.module_manager.scalar_lut_manager.lut.table.to_array()
'''
lut = np.zeros([256,4])

lut[0:90,1]=np.zeros(90)
lut[90:180,1]=np.linspace(1,90,90)
lut[180:,1]=np.zeros(76)+90

lut[0:180,0]=np.zeros(180)
lut[180:,0]=np.linspace(1,76,76)

lut[0:90,2]=np.linspace(0,90,90)
lut[90:,2]=np.zeros(166)+90
'''

lut[:,3]=np.linspace(0,255,256)
lut[0:10,3]=np.zeros(10)+10

surf.module_manager.scalar_lut_manager.lut.table = lut
surf.module_manager.scalar_lut_manager.use_default_range=False

@mlab.animate(delay=100)
def anim():
    f = mlab.gcf()
    for i in range(41):
        array_id = dataset.point_data.add_array(np.ones(10)*i*2)
        dataset.point_data.get_array(array_id).name = 'voltage'
        dataset.point_data.update()
        
        # Updating the dataset
        ms = surf.mlab_source
        ms.m_data.update()

        #surf.mlab_source.voltage = np.ones(10)*i*200
        f.scene.render()
        yield
anim()
mlab.show()
